In [1]:
import pandas as pd
import numpy as np
import random
import json
from datetime import datetime

In [2]:
df_am = pd.read_csv('data/amazon_mobiles/products.csv')
df_a = pd.read_csv('data/amazon/products.csv')
df_e = pd.read_csv('data/ecommerce/products.csv')
df_f = pd.read_csv('data/flipkart/products.csv')
df_j = pd.read_csv('data/jcpenney/products.csv')

dfs = [df_am, df_a, df_e, df_f, df_j]

In [3]:
conj = []
for df in dfs:
    conj += df.to_dict(orient='records')

In [4]:
ids = []
df_am['Id'].apply(lambda x: ids.append((x, 0)))
df_a['Id'].apply(lambda x: ids.append((x, 1)))
df_e['Id'].apply(lambda x: ids.append((x, 2)))
df_f['Id'].apply(lambda x: ids.append((x, 3)))
df_j['Id'].apply(lambda x: ids.append((x, 4)))

0        None
1        None
2        None
3        None
4        None
         ... 
10608    None
10609    None
10610    None
10611    None
10612    None
Name: Id, Length: 10613, dtype: object

In [5]:
df = pd.DataFrame()
df['Id'] = ["purchases/" + str(x) for x in range(1, 100001)]
df['client'] = ["clients/" + str(x) for x in np.random.randint(1, 100000, df.shape[0])]
df['lines'], df['total'], df['orderDate'] = None, None, None

In [6]:
def parseDate(date):
    date = date.replace('Z', '')
    test_date = datetime.strptime(date, "%Y-%m-%d %X")
    return test_date.isoformat()

date_df = pd.read_csv('data/dates.csv')
date_df['date'] = date_df['date'].map(lambda x: parseDate(x))
dates = date_df.date

In [7]:
def makeLine():
    # id = random.choice(ids)
    product = random.choice(conj)
    quantity = random.randint(1, 10)
    return {
        "quantity": quantity,
        "productId": product['Id'],
        "productName": product['name'],
        "unitPrice": product['price'],
        "price": product['price'] * quantity,
        "store": product['store']
    }

def makeOrder():
    lines = [makeLine() for i in range(random.randrange(1, 10))]
    # return [lines, round(sum(line['price'] for line in lines), ndigits=2), random.choice(dates)]
    return {
        "lines": lines,
        "total": round(sum(line['price'] for line in lines), ndigits=2),
        "orderDate": random.choice(dates)
    }

makeOrder()

{'lines': [{'quantity': 1,
   'productId': 'products/jcpenney/7028',
   'productName': 'Alfred Dunner® Chambray Pants - Petite',
   'unitPrice': 53.57,
   'price': 53.57,
   'store': 'jcpenney'},
  {'quantity': 9,
   'productId': 'products/jcpenney/8495',
   'productName': 'Hope Chest 2-Pack Curtain Panels',
   'unitPrice': 223.39,
   'price': 2010.5099999999998,
   'store': 'jcpenney'},
  {'quantity': 1,
   'productId': 'products/jcpenney/1706',
   'productName': 'MAKE UP FOR EVER Chromatic Mix',
   'unitPrice': 207.8,
   'price': 207.8,
   'store': 'jcpenney'},
  {'quantity': 7,
   'productId': 'products/flipkart/169',
   'productName': 'Salt N Pepper 13-019 Femme Taupe Boots Boots',
   'unitPrice': 4195.0,
   'price': 29365.0,
   'store': 'flipkart'}],
 'total': 31636.88,
 'orderDate': '2019-06-19T19:15:22'}

In [8]:
for idx, row in df.iterrows():
    order = makeOrder()
    row['lines'] = order['lines']
    row['total'] = order['total']
    row['orderDate'] = order['orderDate']
    # row['lines'], row['total'], row['orderDate'] = makeOrder()

In [ ]:
result = df.to_json(orient="records", default_handler=str)
parsed = json.loads(result)
json_string = json.dumps(parsed, indent=4)
with open('data/purchases.json', 'w') as outfile:
    outfile.write(json_string)